# Convert foreign payments to USD with historical FX rates

In [1]:
import os
import pandas as pd

from datetime import datetime
from datetime import timedelta

import quandl

In [2]:
if 'QUANDL_KEY' in os.environ:
    quandl.ApiConfig.api_key = os.environ['QUANDL_KEY']

In [3]:
api_call = {
    'ECB': 'ECB/EURUSD',
    'FRED': 'FRED/DEXUSEU',
    'BOE': 'BOE/XUDLSER'
    }

In [4]:
salary = pd.read_csv('example.csv')
salary['date_stamp'] = salary['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
salary

,Date,Amount,date_stamp
0,2017-01-20,1000,2017-01-20
1,2017-02-20,1000,2017-02-20
2,2017-03-20,1500,2017-03-20


In [5]:
date_stamp = salary['date_stamp'][0]
data_source_code = 'ECB'
fx_call_code =api_call[data_source_code]



fx_rate = quandl.get(fx_call_code, start_date=date_stamp, end_date=date_stamp + timedelta(days=1))

In [6]:
def get_fx_rate_smoothed(date_stamp, fx_call_code):
    '''
    Gets fx rate for date according to fx call code, if missing, takes the average
    of fx rates before and after
    '''
    
    try:
        fx_rate = quandl.get(fx_call_code, 
                             start_date=date_stamp, end_date=date_stamp + timedelta(days=1))
    except quandl.errors.NotFoundError as err:
        print(err)
        return
    
    if len(fx_rate) != 0:
        return fx_rate['Value'][0]
    else:
        fx_rate_before = quandl.get(fx_call_code, 
                                    start_date = date_stamp - timedelta(days=7),
                                    end_date = date_stamp - timedelta(days=1))
        fx_rate_after = quandl.get(fx_call_code, start_date=date_stamp + timedelta(days=1), 
                                         end_date=date_stamp + timedelta(days=7))
        
        return (fx_rate_before['Value'][-1] + fx_rate_after['Value'][0])/2
    
get_fx_rate_smoothed(date_stamp, fx_call_code)

1.0632

In [7]:
# For pytest
fx_rate_test = pd.DataFrame(data={'Value':[]})
fx_rate_test.index.rename('Date', inplace=True)
fx_rate_test

,Value
Date,
